In [2]:
#import pandas
import pandas as pd

In [1]:
#Selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

#Other imports
import os 
import wget

In [3]:
# A classe Service é usada para iniciar uma instancia do chrome webdriver
service = Service()

# Webdriver.ChromeOptions é usado para definir a preferencia para o browser chrome
options = webdriver.ChromeOptions()

# Inicia-se a instancia do Chrome WebDriver com as 'options' e 'service'
driver = webdriver.Chrome(service=service, options=options)

In [4]:
link = "https://www.instagram.com/"
driver.get(link)

In [5]:
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

username.clear()
password.clear()
username.send_keys('valim.g')
password.send_keys('V@liminsta2001')

In [6]:
log_in = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [7]:
agora_Nao = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), 'Agora não')]"))).click()
agora_Nao2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Agora não')]"))).click()

In [8]:
profile = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Perfil')]"))).click()

In [9]:
driver.execute_script("window.scrollTo(0,4000);")

images = driver.find_elements(By.CLASS_NAME, "xu96u03")
images = [image.get_attribute('src') for image in images]
images

['https://scontent.cdninstagram.com/v/t51.29350-15/297236900_168784755659440_1865302418720069337_n.webp?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xMDgweDEzNTAuc2RyIn0&_nc_ht=scontent.cdninstagram.com&_nc_cat=104&_nc_ohc=lFmV2LkJbdIAX-jugnC&edm=APs17CUBAAAA&ccb=7-5&ig_cache_key=Mjg5NjY1MTQ4MTA1MjA5NTYwOQ%3D%3D.2-ccb7-5&oh=00_AfC7LzXqn0QjKCK4wrkG6HDb9VQ39oyfcWbN5IrcrS34_A&oe=65EEB38C&_nc_sid=10d13b']

## Salvando as imagens no computador

In [10]:
keyword = "@valim.g"
path = os.getcwd()
path = os.path.join(path, keyword[1:])

os.mkdir(path)
path

'C:\\Users\\Gabriel\\Downloads\\Crawler\\valim.g'

In [11]:
counter = 0
for image in images:
    save_as = os.path.join(path, keyword[1:] + str(counter) + '.jpg')
    wget.download(image, save_as)
    counter += 1

100% [..............................................................................] 99935 / 99935

In [67]:
lista_posts = driver.find_elements(By.CLASS_NAME, "_aabd")
print(len(lista_posts))


0


In [25]:
for elemento in lista_posts:
    link = elemento.find_element(By.TAG_NAME, 'a').get_attribute('href')
    print(link)
    

https://www.instagram.com/unisaojose/p/CzMgkSypOB-/?hl=pt
https://www.instagram.com/unisaojose/p/CzMggUCpD0-/?hl=pt
https://www.instagram.com/unisaojose/p/CzMgaPepGHU/?hl=pt
https://www.instagram.com/unisaojose/p/C3DjkxBOYF8/?hl=pt
https://www.instagram.com/unisaojose/p/C3AqPwnv8p7/?hl=pt
https://www.instagram.com/unisaojose/p/C3AH6T7JA9W/?hl=pt
https://www.instagram.com/unisaojose/reel/C2-F5tGK0kH/?hl=pt
https://www.instagram.com/unisaojose/p/C27S2dwsr8m/?hl=pt
https://www.instagram.com/unisaojose/p/C25QVJcOnqZ/?hl=pt
https://www.instagram.com/unisaojose/p/C22uX-srBOn/?hl=pt
https://www.instagram.com/unisaojose/reel/C20c3h0PZtM/?hl=pt
https://www.instagram.com/unisaojose/p/C20GzustieD/?hl=pt


In [27]:
video_ou_afixada = driver.find_elements(By.CLASS_NAME, "x9bdzbf")
print(len(video_ou_afixada))

8


In [28]:
for item in video_ou_afixada:
    texto = item.get_attribute('aria-label')
    print(texto) 

Carrossel
Clipe
Carrossel
Carrossel
Clipe
Carrossel
None
None
